# Set fold number

In [ ]:
index = 3

# Train

In [ ]:
!cd GWC_YOLOv5 && git checkout master

In [ ]:
import shutil

with open(f"{index}/val.txt") as f:
    for line in f:
        file = "train/" + line.strip()
        shutil.copyfile(file, f"{index}/images/val/" + line.strip())
with open(f"{index}/train.txt") as f:
    for line in f:
        file = "train/" + line.strip()
        shutil.copyfile(file, f"{index}/images/train/" + line.strip())

In [ ]:
import os
import shutil


def run_pseudo_labeling(weights, base_dir, generate=True):
    if generate:
        if os.path.exists('pseudo'):
            shutil.rmtree('pseudo')
        os.mkdir('pseudo')
        os.mkdir('pseudo/images')
        os.mkdir('pseudo/labels')
        os.mkdir('pseudo/images/train')
        os.mkdir('pseudo/labels/train')
        
        for file in os.listdir(base_dir):
            shutil.copyfile(os.path.join(base_dir, file), 'pseudo/labels/train/'+file)
            image = file[:-4] + '.png'
            shutil.copyfile(os.path.join('test', image), 'pseudo/images/train/'+image)

    result = {'train': [f'../{str(index)}/images/train', '../pseudo/images/train'], 'val': f'../{str(index)}/images/val', 'nc': 1, 'names': ['wheat']}
    with open('GWC_YOLOv5/data/custom_pseudo.yaml', 'w') as f:
        dump = yaml.dump(result, default_flow_style=False)
        f.write(dump)
    !cd GWC_YOLOv5 && python train.py --name final --img 800 --batch 8 --epochs 15 --data custom_pseudo.yaml --weights {weights}  --hyp data/hyp.finetune_small_lr.yaml --cache

In [ ]:
run_pseudo_labeling(
    f"runs/train/4fold{index}_pseudo_master/weights/best.pt",
    "labels",
)

In [ ]:
# save to submission
!cd GWC_YOLOv5 && python detect.py --img-size 800 --name submission --weights ../runs/train/final/weights/best.pt --source ../test --augment --nosave --save-csv --conf-thres 0.50